# **ResNet**
此份程式碼會介紹如何使用 torch.nn 的方式建構 ResNet 的模型架構。

![image](https://hackmd.io/_uploads/rJ8SA1HOp.png)

- [source paper](https://arxiv.org/abs/1512.03385)

## 匯入套件

In [1]:
# PyTorch 相關套件
import torch
import torch.nn as nn

## ResNet Arhietecture

![image](https://hackmd.io/_uploads/B16H0kHOT.png)

- [source paper](https://arxiv.org/abs/1512.03385)

In [18]:
class BasicBlock(nn.Module):
    def __init__(self, input_channels, output_channels, kernel_size=3, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, output_channels, kernel_size, stride, padding=1)
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.conv2 = nn.Conv2d(output_channels, output_channels, kernel_size, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(output_channels)
        self.act = nn.ReLU()
        self.stride = stride
        # 如果strides不等於1，則需要對identity進行下採樣
        if self.stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv2d(input_channels, output_channels, 1, stride),
                nn.BatchNorm2d(output_channels)
            )
        else:
            self.downsample = None
        
        
    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act(x)
        x = self.conv2(x)
        x = self.bn2(x)
        
        if self.downsample is not None:
            identity = self.downsample(identity)
        
        x = x + identity
        x = self.act(x)
        return x
            

In [19]:
class ResBlock(nn.Module):
    def __init__(self, input_channels, output_channels, layers, strides=1):
        super().__init__()
        layers_list = []
        if strides != 1:
            layers_list.append(BasicBlock(input_channels, output_channels, 3, strides))
        for _ in range(1, layers):
            layers_list.append(BasicBlock(output_channels, output_channels, 3))
        self.layers = nn.Sequential(*layers_list)
    def forward(self, x):
        return self.layers(x)

In [22]:
class ResNet(nn.Module):
    def __init__(self, num_classes, layers):
        super().__init__()
        # stem
        self.conv1 = nn.Conv2d(3, 64, 7, 2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.act = nn.ReLU()
        self.maxpool = nn.MaxPool2d(3, 2, padding=1)
        
        self.layer1 = ResBlock(64, 64, layers=layers[0])
        self.layer2 = ResBlock(64, 128, layers=layers[1], strides=2)
        self.layer3 = ResBlock(128, 256, layers=layers[2], strides=2)
        self.layer4 = ResBlock(256, 512, layers=layers[3], strides=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [23]:
# ResNet-18
model = ResNet(10, [2, 2, 2, 2])
# ResNet-34
model = ResNet(10, [3, 4, 6, 3])

inputs = torch.randn(1, 3, 224, 224)
outputs = model(inputs)
print(outputs.size())

torch.Size([1, 10])
